In [ ]:
#Detecting the files with double extension (Example: "file_name.dcm.raw")
def clean_raw_extensions(main_path, patient_folders):
    for patient in patient_folders:
        patient_images = os.listdir(main_path+patient)
        for patient_image in patient_images:
            extension = patient_image[-5::1]

            #If the last 5 position of the files are equal to "m.raw", the extension will be changed.
            if extension == "m.raw":
                complete_path = main_path+patient+'/'+patient_image
                print(complete_path)
                fix_path = complete_path[:-4]
                os.rename(complete_path, fix_path)

#Detecting the files that are automatically created by MACOS (.DS_Store)
def delete_mac_files(main_path, patient_folders):
    for patient in patient_folders:
        patient_images = os.listdir(main_path+patient)
        for patient_image in patient_images:
            complete_path = main_path+patient+'/'+patient_image
            file_name, file_extension = os.path.splitext(complete_path)

            #If the extension of the file is ".DS_Store", the file will be removed.
            if file_extension == ".DS_Store":
                print(complete_path)
                os.remove(complete_path)               

clean_raw_extensions(main_path, patient_folders)
delete_mac_files(main_path, patient_folders)

Volumes .DCM images

In [ ]:
test_patient = '/Users/alechiquito/Documents/osic_db/images/ID00296637202279895784347'
vol = imageio.volread(test_patient, 'DICOM')

print('Shape of image array:', vol.shape)

fig, axes = plt.subplots(figsize=(10,10), nrows=1, ncols=4)

[axes[x].imshow(vol[x], cmap='gray') for x in range(4)]
[ax.axis('off') for ax in axes]

#axes[0].imshow(vol[0,:,:], cmap='gray')
#axes[1].imshow(vol[10,:,:], cmap='gray')
#axes[2].imshow(vol[15,:,:], cmap='gray')
#axes[3].imshow(vol[26,:,:], cmap='gray')


plt.show()

In [ ]:
def show_slices_mask(main_path, patient):
    main_path = main_path+patient

    vol = imageio.volread(main_path, 'DICOM')
    print('Shape of image array:', vol.shape)

    num_slices = vol.shape[0]
    fig, axes = plt.subplots(figsize=(10,6), nrows=3, ncols=5)

    img_list = np.linspace(0,num_slices,12, dtype=int)
    print(img_list)

    for i, ax in enumerate(axes.ravel()):
        ax.imshow(vol[i,:,:], cmap='gray')
        ax.axis('off')

    plt.show()
    

show_slices_mask('ID00296637202279895784347')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Usar una red neuronal preentrenada (MobileNetV2 como ejemplo)
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 1))
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D()
])

# Preparar datos de entrada
transform = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = transform.flow_from_directory('path/to/your/dataset', target_size=(128, 128), batch_size=32, class_mode=None, shuffle=False)

# Extraer características
features = model.predict(test_generator)

# Aplicar K-means a las características extraídas
features_flat = features.reshape(features.shape[0], -1)
kmeans = KMeans(n_clusters=10, random_state=0).fit(features_flat)
labels = kmeans.labels_

### Resize and normalize images

In [ ]:
image_pydicom = pydicom.dcmread(path_images[0]).pixel_array
img_size = image_pydicom.shape
print(img_size)

# Resize the image to 512x512
resized_image = cv2.resize(image_pydicom, (224, 224))
print(resized_image.shape)
print(type(resized_image), type(image_pydicom))

#Normalize the image 
normalized_image = cv2.normalize(resized_image, None, 0, 255, cv2.NORM_MINMAX)

# Plot the original and resized images
plt.figure(figsize=(15, 15))
plt.subplot(1, 3, 1)
plt.imshow(image_pydicom, cmap='gray')
plt.title('Original Image')
#plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(resized_image, cmap='gray')
plt.title('Resized (224x224)')
#plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(normalized_image, cmap='gray')
plt.title('Normalized (0-255)')
#plt.axis('off')
plt.show()

In [ ]:
import cv2
# Cargar la imagen usando OpenCV
imagen = cv2.imread(path_images[0])

# Convertir la imagen de BGR a RGB
imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

# Separar los canales
canal_rojo, canal_verde, canal_azul = cv2.split(imagen)

# Crear una figura con subgráficas
plt.figure(figsize=(10, 5))

# Mostrar el canal rojo
plt.subplot(1, 3, 1)
plt.imshow(canal_rojo, cmap='Reds')
plt.title('Canal Rojo')
plt.axis('off')

# Mostrar el canal verde
plt.subplot(1, 3, 2)
plt.imshow(canal_verde, cmap='Greens')
plt.title('Canal Verde')
plt.axis('off')

# Mostrar el canal azul
plt.subplot(1, 3, 3)
plt.imshow(canal_azul, cmap='Blues')
plt.title('Canal Azul')
plt.axis('off')

# Mostrar la figura
plt.tight_layout()
plt.show()


In [ ]:
import tensorflow as tf
from sklearn.cluster import KMeans
import numpy as np

# Load a pre-trained model (ResNet50)
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Preprocess the input data (e.g., CIFAR-10)
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
x_train = tf.keras.applications.resnet50.preprocess_input(x_train)
x_test = tf.keras.applications.resnet50.preprocess_input(x_test)

# Extract features
train_features = base_model.predict(x_train)
test_features = base_model.predict(x_test)

# Apply K-means clustering
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(train_features)
cluster_labels = kmeans.predict(test_features)


In [ ]:
#Print the result of loading the images
print(type(images[0]))
print(images[0].shape)
#images[0]

#Convert from list to numpy array
print(type(images))
images = np.array(images)
images = images[:, :, :, :3]
print(type(images), images.shape)

In [ ]:
#Create a list with the patient ids folders
patient_folders = os.listdir(train_main_path)

for patient_folder in patient_folders:
    if patient_folder.startswith('._'):
        patient_folders.remove(patient_folder)

#Get the array of every image and add it to a list
path_images = []

for patient in patient_folders:
    if patient.startswith('._'):
        pass
    else:
        temp_path = train_main_path+patient+'/'
        images = os.listdir(train_main_path+patient)
        for image in images:
            if image.startswith('._'):
                pass
            else:
                temp_path_image = temp_path + image
                path_images.append(temp_path_image)

print(len(patient_folders))
print(len(path_images))
print(path_images[0])
patient_folders

In [ ]:
images = []

for image_batch in dataset:
    images.append(image_batch.numpy())

images = np.concatenate(images)
print(images.shape)